In [2]:
import numpy as np
import sympy
from collections import OrderedDict

In [3]:
import qsymm

# Graphene

We use the Hamiltonian generator to reproduce the spinless nearest neighbour tight binding Hamiltonian for graphene.

The generators of the symmetry group are time-reversal symmetry, sublattice (or chiral) symmetry, and threefold rotation symmetry.

In [4]:
# Time reversal
TR = qsymm.time_reversal(2, U=np.eye(2))

# Chiral symmetry
C = qsymm.chiral(2, U=np.array([[1, 0], [0, -1]]))

# Atom A rotates into A, B into B, use exact sympy representation
sphi = 2*sympy.pi/3
RC3 = sympy.Matrix([[sympy.cos(sphi), -sympy.sin(sphi)],
                  [sympy.sin(sphi), sympy.cos(sphi)]])
C3 = qsymm.PointGroupElement(RC3, False, False, np.eye(2))

symmetries = [C, TR, C3]

There are two carbon atoms per unit cell (A and B) with one orbital each. The lattice is triangular, and only includes hoppings between nearest neighbour atoms. This restricts hoppings to only those between atoms of different types, such that each atom couples to three neighbouring atoms. Using the symmetrization strategy to generate the Hamiltonian, it is sufficient to specify hoppings to one such neighbour along with the symmetry generators, and we take the vector $(1, 0)$ to connect this neighbouring pair of atoms.

In [5]:
norbs = OrderedDict([('A', 1), ('B', 1)])  # A and B atom per unit cell, one orbital each
hopping_vectors = [('A', 'B', [0, 1])] # Hopping between neighbouring A and B atoms

Generate the Hamiltonian.

In [6]:
family = qsymm.bloch_family(hopping_vectors, symmetries, norbs)

In [7]:
qsymm.display_family(family)

Matrix([
[                                                                                   0, e**(I*k_y) + e**(-I*k_y/2)*e**(sqrt(3)*I*k_x/2) + e**(-I*k_y/2)*e**(-sqrt(3)*I*k_x/2)],
[e**(I*k_y/2)*e**(sqrt(3)*I*k_x/2) + e**(I*k_y/2)*e**(-sqrt(3)*I*k_x/2) + e**(-I*k_y),                                                                                     0]])

Scale the bond length in terms of the graphene lattice constant, and have the function return a list of BlochModel objects. For this we can use a more convenient definition of the rotation

In [104]:
C3 = qsymm.rotation(1/3, U=np.eye(2))
symmetries = [C, TR, C3]

In [105]:
norbs = OrderedDict([('A', 1), ('B', 1)])  # A and B atom per unit cell, one orbital each
hopping_vectors = [('A', 'B', [0, 1/np.sqrt(3)])] # Hopping between neighbouring A and B atoms

In [106]:
family = qsymm.bloch_family(hopping_vectors, symmetries, norbs, bloch_model=True)

In [107]:
qsymm.display_family(family)

Matrix([
[                                                                                             0, e**(I*k_x/2)*e**(-sqrt(3)*I*k_y/6) + e**(sqrt(3)*I*k_y/3) + e**(-I*k_x/2)*e**(-sqrt(3)*I*k_y/6)],
[e**(I*k_x/2)*e**(sqrt(3)*I*k_y/6) + e**(-sqrt(3)*I*k_y/3) + e**(-I*k_x/2)*e**(sqrt(3)*I*k_y/6),                                                                                               0]])

# Three-orbital tight binding model for monolayer MX$_2$

We use the Hamiltonian generator to reproduce the tight binding model for monolayer MX$_2$ published in Phys. Rev. B 88, 085433 (2013).

The generators of the symmetry group of the tight binding model are time reversal symmetry, mirror symmetry and threefold rotation symmetry.

In [ ]:
# Time reversal
TR = qsymm.time_reversal(2, np.eye(3))

# Mirror symmetry 
Mx = qsymm.mirror([1, 0], np.diag([1, -1, 1]))

# Threefold rotation on d_z^2, d_xy, d_x^2-y^2 states.
C3U = np.array([[1, 0, 0],
                 [0, -0.5, -np.sqrt(3)/2],
                 [0, np.sqrt(3)/2, -0.5]])
# Could also use the predefined representation of rotations on d-orbitals
Ld = qsymm.groups.L_matrices(3, 2)
C3U2 = qsymm.groups.spin_rotation(2 * np.pi * np.array([0, 0, 1/3]), Ld)
# Restrict to d_z^2, d_xy, d_x^2-y^2 states
mask = np.array([1, 2 ,0])
C3U2 = C3U2[mask][:, mask]
assert np.allclose(C3U, C3U2)

C3 = qsymm.rotation(1/3, U=C3U)

symmetries = [TR, Mx, C3]

Next, we specify the hoppings to include. The tight binding model has a triangular lattice, three orbitals per M atom, and nearest neighbour hopping.

In [ ]:
# One site per unit cell (M atom), with three orbitals
norbs = OrderedDict([('a', 3)])

Each atom has six nearest neighbour atoms at a distance of one primitive lattice vector. Since we use the symmetrization strategy to generate the Hamiltonian, it is sufficient to specify a hopping to one nearest neighbour atom along with the symmetry generators. We take the primitive vector connecting the pair of atoms to be $(1, 0)$.

In [ ]:
# Hopping to a neighbouring atom one primitive lattice vector away
hopping_vectors = [('a', 'a', [1, 0])]

Generate the tight binding Hamiltonian.

In [ ]:
family = qsymm.bloch_family(hopping_vectors, symmetries, norbs, bloch_model=True)

The Hamiltonian family should include 8 linearly independent components, including the onsite terms.

In [ ]:
len(family)

In [ ]:
qsymm.display_family(family, nsimplify=True)

# 4-site model for monolayer WTe$_2$

We use the Hamiltonian generator to reproduce the tight binding model for monolayer WTe$_2$ published in Phys. Rev. X 6, 041069 (2016).

The generators of the symmetry group of the tight binding model are time reversal symmetry, glide reflection and inversion symmetry.

In [ ]:
# Define 4 sites with one orbital each
sites = ['Ad', 'Ap', 'Bd', 'Bp']
norbs = OrderedDict([(site, 1) for site in sites])

# Define symbolic coordinates for orbitals
rAp = qsymm.sympify('[x_Ap, y_Ap]')
rAd = qsymm.sympify('[x_Ad, y_Ad]')
rBp = qsymm.sympify('[x_Bp, y_Bp]')
rBd = qsymm.sympify('[x_Bd, y_Bd]')

# Define hoppings to include
hopping_vectors = [('Bd', 'Bd', np.array([1, 0])),
                   ('Ap', 'Ap', np.array([1, 0])),
                   ('Bd', 'Ap', rAp - rBd),
                   ('Ap', 'Bp', rBp - rAp),
                   ('Ad', 'Bd', rBd - rAd),                   
                  ]

In [ ]:
# Inversion
perm_inv = {'Ad': 'Bd', 'Ap': 'Bp', 'Bd': 'Ad', 'Bp': 'Ap'}
onsite_inv = {site: (1 if site in ['Ad', 'Bd'] else -1) * np.eye(1) for site in sites}
inversion = qsymm.groups.symmetry_from_permutation(-np.eye(2), perm_inv, norbs, onsite_inv)

# Glide
perm_glide = {site: site for site in sites}
onsite_glide = {site: (1 if site in ['Ad', 'Bd'] else -1) * np.eye(1) for site in sites}
glide = qsymm.groups.symmetry_from_permutation(np.array([[-1, 0],[0, 1]]), perm_glide, norbs, onsite_glide)

# TR
time_reversal = qsymm.time_reversal(2, np.eye(4))

gens = {glide, inversion, time_reversal}
sg = qsymm.groups.generate_group(gens)
len(sg)

Generate the tight binding Hamiltonian.

In [ ]:
family = qsymm.bloch_family(hopping_vectors, gens, norbs=norbs)
print(len(family), [len(fam) for fam in family])

In [ ]:
qsymm.display_family(family)

# Square lattice with 4 sites in the UC

Make model with square lattice that has 4 sites in the unit cell related by 4-fold rotation. Sites have spin-1/2 and we add time reversal and particle-hole symmetry.

In [ ]:
hopping_vectors = [('a', 'b', np.array([1, 0])),
                   ('b', 'a', np.array([1, 0])),
                   ('c', 'd', np.array([1, 0])),
                   ('d', 'c', np.array([1, 0])),
                   ('a', 'c', np.array([0, 1])),
                   ('c', 'a', np.array([0, 1])),
                   ('b', 'd', np.array([0, 1])),
                   ('d', 'b', np.array([0, 1])),                  
                  ]

# Define spin-1/2 operators
S = qsymm.groups.spin_matrices(1/2)
# Define real space rotation generators in 2D
L = qsymm.groups.L_matrices(d=2)

sites = ['a', 'b', 'c', 'd']
norbs = OrderedDict([(site, 2) for site in sites])

perm_C4 = {'a': 'b', 'b': 'd', 'd': 'c', 'c': 'a'}
onsite_C4 = {site: qsymm.groups.spin_rotation(2*np.pi * np.array([0, 0, 1/4]), S) for site in sites}
C4 = qsymm.groups.symmetry_from_permutation(
                qsymm.groups.spin_rotation(2*np.pi * np.array([1/4]), L, roundint=True), 
                perm_C4, norbs, onsite_C4)

# Fermionic TR
time_reversal = qsymm.time_reversal(2, 
                np.kron(np.eye(4), qsymm.groups.spin_rotation(2*np.pi * np.array([0, 1/2, 0]), S)))

# define strange PH symmetry
particle_hole = qsymm.particle_hole(2, np.eye(8))

gens = {C4, time_reversal, particle_hole}

sg = qsymm.groups.generate_group(gens)
len(sg)

In [ ]:
family = qsymm.bloch_family(hopping_vectors, gens, norbs=norbs)
print(len(family), [len(fam) for fam in family])
qsymm.display_family(family)

### C4T symmetric model

In [27]:
hopping_vectors = [('a', 'a', np.array([1, 0])),               
                  ]

# Define spin-1/2 operators
S = qsymm.groups.spin_matrices(1/2)
# Define real space rotation generators in 2D
L = qsymm.groups.L_matrices(d=2)

sites = ['a']
norbs = OrderedDict([(site, 4) for site in sites])

perm_C4 = {'a': 'a'}
onsite_C4 = {site: np.kron(np.eye(2),
                            qsymm.groups.spin_rotation(2*np.pi * np.array([0, 0, 1/4]), S)
                            )
              for site in sites}
C4 = qsymm.groups.symmetry_from_permutation(
                qsymm.groups.spin_rotation(2*np.pi * np.array([1/4]), L, roundint=True), 
                perm_C4, norbs, onsite_C4)

# Fermionic TR
T = qsymm.time_reversal(2, 
                np.kron(np.eye(2),
                        qsymm.groups.spin_rotation(2*np.pi * np.array([0, 1/2, 0]), S),
                        )
                       )

C4T = C4 * T

sg = qsymm.groups.generate_group(gens)
len(sg)

8

In [28]:
sg

{1, R(π), R(π/2), R(-π/2), T, R(π) T, R(π/2) T, R(-π/2) T}

In [30]:
familyTR = qsymm.bloch_family(hopping_vectors, {T, C4}, norbs=norbs)
print(len(familyTR), [len(fam) for fam in familyTR])
qsymm.display_family(familyTR)

14 [1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0],
[0, 1, 0, 0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

Matrix([
[ 0, 0, I,  0],
[ 0, 0, 0, -I],
[-I, 0, 0,  0],
[ 0, I, 0,  0]])

...

Matrix([
[0, 0,                                                       0,                                                       0],
[0, 0,                                                       0,                                                       0],
[0, 0,                                                       0, I*e**(I*k_x) + e**(I*k_y) - e**(-I*k_y) - I*e**(-I*k_x)],
[0, 0, I*e**(I*k_x) - e**(I*k_y) + e**(-I*k_y) - I*e**(-I*k_x),                                                       0]])

Matrix([
[                                                      0, -e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x), 0, 0],
[e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x),                                                        0, 0, 0],
[                                                      0,                                                        0, 0, 0],
[                                                      0,                                                        0, 0, 0]])

Matrix([
[                                                           0,                                                           0, I*e**(I*k_x) + I*e**(I*k_y) + I*e**(-I*k_y) + I*e**(-I*k_x),                                                            0],
[                                                           0,                                                           0,                                                           0, -I*e**(I*k_x) - I*e**(I*k_y) - I*e**(-I*k_y) - I*e**(-I*k_x)],
[-I*e**(I*k_x) - I*e**(I*k_y) - I*e**(-I*k_y) - I*e**(-I*k_x),                                                           0,                                                           0,                                                            0],
[                                                           0, I*e**(I*k_x) + I*e**(I*k_y) + I*e**(-I*k_y) + I*e**(-I*k_x),                                                           0,                                                            0]]

Matrix([
[                                                      0,                                                        0,                                                       0, -e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x)],
[                                                      0,                                                        0, e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x),                                                        0],
[                                                      0, -e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x),                                                       0,                                                        0],
[e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x),                                                        0,                                                       0,                                                        0]])

Matrix([
[0, 0,                                                       0,                                                        0],
[0, 0,                                                       0,                                                        0],
[0, 0,                                                       0, -e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x)],
[0, 0, e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x),                                                        0]])

In [31]:
familyC4T = qsymm.bloch_family(hopping_vectors, {C4T}, norbs=norbs)
print(len(familyC4T), [len(fam) for fam in familyC4T])
qsymm.display_family(familyC4T)

20 [1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0],
[0, 1, 0, 0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

Matrix([
[ 0, 0, I,  0],
[ 0, 0, 0, -I],
[-I, 0, 0,  0],
[ 0, I, 0,  0]])

...

Matrix([
[                            0,                            0, I*e**(I*k_y) + I*e**(-I*k_y),                             0],
[                            0,                            0,                            0, -I*e**(I*k_x) - I*e**(-I*k_x)],
[-I*e**(I*k_y) - I*e**(-I*k_y),                            0,                            0,                             0],
[                            0, I*e**(I*k_x) + I*e**(-I*k_x),                            0,                             0]])

Matrix([
[                           0,                         0,                        0, I*e**(I*k_y) - I*e**(-I*k_y)],
[                           0,                         0, e**(I*k_x) - e**(-I*k_x),                            0],
[                           0, -e**(I*k_x) + e**(-I*k_x),                        0,                            0],
[I*e**(I*k_y) - I*e**(-I*k_y),                         0,                        0,                            0]])

Matrix([
[                       0,                            0,                            0, -e**(I*k_x) + e**(-I*k_x)],
[                       0,                            0, I*e**(I*k_y) - I*e**(-I*k_y),                         0],
[                       0, I*e**(I*k_y) - I*e**(-I*k_y),                            0,                         0],
[e**(I*k_x) - e**(-I*k_x),                            0,                            0,                         0]])

Matrix([
[                           0,                             0, -I*e**(I*k_x) - I*e**(-I*k_x),                            0],
[                           0,                             0,                             0, I*e**(I*k_y) + I*e**(-I*k_y)],
[I*e**(I*k_x) + I*e**(-I*k_x),                             0,                             0,                            0],
[                           0, -I*e**(I*k_y) - I*e**(-I*k_y),                             0,                            0]])

Matrix([
[0, 0,                                                       0,                                                        0],
[0, 0,                                                       0,                                                        0],
[0, 0,                                                       0, -e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x)],
[0, 0, e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x),                                                        0]])

In [32]:
family_new = qsymm.hamiltonian_generator.subtract_family(familyC4T, familyTR, prettify=True)
print(len(family_new), [len(fam) for fam in family_new])
qsymm.display_family(family_new)

6 [4, 4, 4, 4, 4, 4]


Matrix([
[-e**(I*k_x) + e**(I*k_y) + e**(-I*k_y) - e**(-I*k_x),                                                   0, 0, 0],
[                                                   0, e**(I*k_x) - e**(I*k_y) - e**(-I*k_y) + e**(-I*k_x), 0, 0],
[                                                   0,                                                   0, 0, 0],
[                                                   0,                                                   0, 0, 0]])

Matrix([
[                                                   0,                                                   0, -e**(I*k_x) + e**(I*k_y) + e**(-I*k_y) - e**(-I*k_x),                                                   0],
[                                                   0,                                                   0,                                                    0, e**(I*k_x) - e**(I*k_y) - e**(-I*k_y) + e**(-I*k_x)],
[-e**(I*k_x) + e**(I*k_y) + e**(-I*k_y) - e**(-I*k_x),                                                   0,                                                    0,                                                   0],
[                                                   0, e**(I*k_x) - e**(I*k_y) - e**(-I*k_y) + e**(-I*k_x),                                                    0,                                                   0]])

Matrix([
[                                                       0,                                                       0,                                                       0, -I*e**(I*k_x) + e**(I*k_y) - e**(-I*k_y) + I*e**(-I*k_x)],
[                                                       0,                                                       0, I*e**(I*k_x) + e**(I*k_y) - e**(-I*k_y) - I*e**(-I*k_x),                                                        0],
[                                                       0, I*e**(I*k_x) - e**(I*k_y) + e**(-I*k_y) - I*e**(-I*k_x),                                                       0,                                                        0],
[-I*e**(I*k_x) - e**(I*k_y) + e**(-I*k_y) + I*e**(-I*k_x),                                                       0,                                                       0,                                                        0]])

Matrix([
[0, 0,                                                    0,                                                   0],
[0, 0,                                                    0,                                                   0],
[0, 0, -e**(I*k_x) + e**(I*k_y) + e**(-I*k_y) - e**(-I*k_x),                                                   0],
[0, 0,                                                    0, e**(I*k_x) - e**(I*k_y) - e**(-I*k_y) + e**(-I*k_x)]])

Matrix([
[                                                          0,                                                           0, -I*e**(I*k_x) + I*e**(I*k_y) + I*e**(-I*k_y) - I*e**(-I*k_x),                                                            0],
[                                                          0,                                                           0,                                                            0, -I*e**(I*k_x) + I*e**(I*k_y) + I*e**(-I*k_y) - I*e**(-I*k_x)],
[I*e**(I*k_x) - I*e**(I*k_y) - I*e**(-I*k_y) + I*e**(-I*k_x),                                                           0,                                                            0,                                                            0],
[                                                          0, I*e**(I*k_x) - I*e**(I*k_y) - I*e**(-I*k_y) + I*e**(-I*k_x),                                                            0,                                                            0]]

Matrix([
[                                                       0,                                                        0,                                                       0, e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) - e**(-I*k_x)],
[                                                       0,                                                        0, e**(I*k_x) - I*e**(I*k_y) + I*e**(-I*k_y) - e**(-I*k_x),                                                       0],
[                                                       0, -e**(I*k_x) - I*e**(I*k_y) + I*e**(-I*k_y) + e**(-I*k_x),                                                       0,                                                       0],
[-e**(I*k_x) + I*e**(I*k_y) - I*e**(-I*k_y) + e**(-I*k_x),                                                        0,                                                       0,                                                       0]])

In [37]:
from scipy import linalg as la
la.det(np.kron(np.eye(4), 2 * S[1]))

(1-0j)

### Find periods from BlochModel

In [108]:
ham = qsymm.hamiltonian_generator.hamiltonian_from_family(family, tosympy=False)

In [109]:
ham = qsymm.BlochModel(ham)
ham

{(array([0.5       , 0.28867513]), c0):
[[0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j]],

(array([ 0.5       , -0.28867513]), c0):
[[0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j]],

(array([-0.5       , -0.28867513]), c0):
[[0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j]],

(array([-0.        ,  0.57735027]), c0):
[[0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j]],

(array([-0.5       ,  0.28867513]), c0):
[[0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j]],

(array([-0.        , -0.57735027]), c0):
[[0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j]],

}

In [148]:
from collections import defaultdict
from qsymm.model import BlochModel, BlochCoeff
from qsymm.kwant_linalg_lll import lll
from itertools import product
from functools import reduce
from math import gcd
# from https://github.com/lan496/hsnf
from hsnf import smith_normal_form, row_style_hermite_normal_form

one = sympy.sympify(1)

def model_periods(model):
    """Find the periods and positions of orbitals
    for the lattice model where it is is periodic.
    It needs to be a bloch Hamiltonian written in the
    real space convention (i.e. hopping phases correspond
    to the real space sepatarion of orbitals)"""
    # convert to BlochModel
    model = BlochModel(model)
    n = model.shape[0]
    d = len(model.momenta)

    # dictionary of relative positions for each pair of
    # orbitals, allow multiple relative positions
    rel_pos_nn = defaultdict(lambda: set())
    # populate it with all the hoppings in the model
    for (hop, _), val in model.items():
        for a, b in zip(*np.nonzero(val)):
            rel_pos_nn[a, b].add(BlochCoeff(hop, one))

    # Keep adding further neighbor relative positions up to n steps,
    # this guarantees we find all linearly independent (over integers)
    # lattice vectors.
    # This may take long if there are many orbitals, perhaps there is
    # a better stopping condition.
    rel_pos = defaultdict(lambda: set(), {ind: set.copy() for ind, set in rel_pos_nn.items()})
    for _ in range(n-1):
        for ((a1, b1), hops1), ((a2, b2), hops2) in product(rel_pos.items(), rel_pos_nn.items()):
            if b1 == a2:
                for (hop1, hop2) in product(hops1, hops2):
                    rel_pos[a1, b2].add(hop1 * hop2)

    # Lattice vectors are those connecting the same site type
    vecs = np.array([vec for ((a, b), hops) in rel_pos.items() if a == b
                         for vec, _ in hops])
    # Find primitive vectors
    prim_vecs = primitive_lattice_vecs(vecs)
    return prim_vecs


def make_int(R):
    # If close to an integer array convert to integer array, else
    # return None
    R_int = np.array(np.round(R), int)
    if qsymm.linalg.allclose(R, R_int):
        return R_int
    else:
        return None


def primitive_lattice_vecs(vecs, method='hnf'):
    """Find a set of primitive lattice vectors spanning the
    same lattice as vecs."""
    vecs = np.asarray(vecs)
    d = vecs.shape[1]
    # pick out a set of small linearly independent vectors
    ### TODO this likely works even if vecs don't span the full space
    norms = np.linalg.norm(vecs, axis=1)
    vecs = vecs[np.argsort(norms)]
    bas = []
    for v in vecs:
        new_bas = bas + [v]
        rank = np.linalg.matrix_rank(new_bas)
        if rank == len(new_bas):
            bas = new_bas
        if rank == d:
            break
    else:
        raise ValueError('The vectors do not span the full space.')
    bas = np.vstack(bas)
    # rewrite all vectors in this new basis
    vecs = np.linalg.solve(bas.T, vecs.T).T
    # find the common denominator to make all integer
    denom = lcd(vecs)
    vecs = make_int(vecs * denom)
    bas = bas / denom
    # find primitive lattice vectors
    if method == 'snf':
        D, L, R = smith_normal_form(vecs)
        prim_vecs = D @ np.linalg.inv(R)
    elif method == 'hnf':
        prim_vecs, U = row_style_hermite_normal_form(vecs)
    else:
        raise ValueError("Method must be either 'hnf' or 'snf'.")
    prim_vecs = prim_vecs[:d]
    # convert back to original basis
    prim_vecs = prim_vecs @ bas
    return lll(prim_vecs)[0]


def lcd(a, tol=1e-9):
    """Calculate approximate least common denominator for
    the entries of floating point array `a`."""
    a = np.asarray(a).flatten()
    a = a % 1
    ### This would be nice to vectorize
    denoms = np.array([farey(x)[1] for x in a])
    return reduce(lambda a, b: a * b // gcd(a, b), denoms, 1)
    

def farey(x, tol=1e-9):
    """Farey sequence rational approximation of `0 <= x <= 1`
    with tolerance `tol`. Adapted from
    https://www.johndcook.com/blog/2010/10/20/best-rational-approximation/
    """
    N = int(tol**(-1/2) / np.sqrt(5))
    a, b = 0, 1
    c, d = 1, 1
    while (b <= N and d <= N):
        # Add these checks to avoid additional loops
        if abs(x - a/b) <= tol:
            return a, b
        elif abs(x - c/d) <= tol:
            return c, d
        mediant = (a+c)/(b+d)
        if abs(x - mediant) <= tol:
            return a+c, b+d
        elif x > mediant:
            a, b = a+c, b+d
        else:
            c, d = a+c, b+d

    if (b > N):
        return c, d
    else:
        return a, b

In [149]:
# %%time
%timeit lat_vecs = model_periods(ham)
lat_vecs

55.8 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


array([[ 1.00000000e+00, -2.77555756e-16],
       [ 5.00000000e-01,  8.66025404e-01]])

In [114]:
%load_ext line_profiler

In [150]:
%lprun -f farey model_periods(ham)

Timer unit: 1e-06 s

Total time: 0.000181 s
File: <ipython-input-148-82fb982a8674>
Function: farey at line 111

Line #      Hits         Time  Per Hit   % Time  Line Contents
   111                                           def farey(x, tol=1e-9):
   112                                               """Farey sequence rational approximation of `0 <= x <= 1`
   113                                               with tolerance `tol`. Adapted from
   114                                               https://www.johndcook.com/blog/2010/10/20/best-rational-approximation/
   115                                               """
   116        28         81.0      2.9     44.8      N = int(tol**(-1/2) / np.sqrt(5))
   117        28         13.0      0.5      7.2      a, b = 0, 1
   118        28         11.0      0.4      6.1      c, d = 1, 1
   119        28         13.0      0.5      7.2      while (b <= N and d <= N):
   120                                                   # Add these checks

In [88]:
%%time
primitive_lattice_vecs([[11, 1], [1, 0], [0, 0]], method='hnf')

CPU times: user 142 ms, sys: 24 µs, total: 142 ms
Wall time: 138 ms


array([[1., 0.],
       [0., 1.]])